In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install catboost
!pip install xgboost

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Explore the data**

In [ ]:
data = pd.read_csv("/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv")
data

In [ ]:
data.describe().T

In [ ]:
data["age"].plot(kind = "hist")

In [ ]:
# corr_matrix = data.corr().T

# from pandas.plotting import scatter_matrix

# scatter_matrix(data)
data.corr().T

# **Set features and lables**

In [ ]:
# target and feature data
y = data["output"].copy()
X = data.copy()
del X["output"]
# del X["fbs"]
# del X["chol"]
X

# **Train, test, split and assign classifier**

In [ ]:
# Train test and split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.40, random_state = 32)

In [ ]:
# clf = DecisionTreeClassifier(max_leaf_nodes = 5, random_state = 80) = 85%
clf = CatBoostClassifier(verbose=0, n_estimators=40)
clf.fit(X_train, y_train)

# **Predictions**

In [ ]:
predictions = clf.predict(X_test)

In [ ]:
actuals = np.ceil(list(y_test))

for prediction,actual in list(zip(predictions, actuals))[:10]:
    print("Prediction:", int(prediction), "Actual:", int(actual))

# **Confusion matrix**

In [ ]:
import seaborn as sn

confusion_matrix = pd.crosstab(actuals, predictions, rownames=['Actual'], colnames=['Predicted'])

sn.heatmap(confusion_matrix, annot=True)

confusion_matrix

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(actuals, predictions)
confusion_matrix

In [ ]:
# Get confusion matrix metrics
true_positives = int(confusion_matrix[:1,:1])
true_negatives = int(confusion_matrix[-1:, -1:])
false_positives = int(confusion_matrix[:1,-1:])
false_negatives = int(confusion_matrix[-1:, :-1])

In [ ]:
# Sensitivity
sensitivity = round(true_positives / (true_positives + false_negatives), 2)
sensitivity

In [ ]:
# Specificity
specificity = round(true_negatives / (true_negatives + false_positives), 2)
specificity

In [ ]:
# Precision
precision = round(true_positives / (true_positives + false_positives),2)
precision

# **Accuracy : 88%**
# **F1 Score : 87%**

In [ ]:
# Accuracy
round(accuracy_score(predictions, y_test), 2)

In [ ]:
# F1 score
round(2*(specificity * precision) / (specificity + precision), 2)